<a href="https://colab.research.google.com/github/huynguyenfts/Example-sentiment-comment/blob/master/Example_Sentiment_Comment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Clone git repo **

In [0]:
!pip install -q xlrd
!git clone https://github.com/huynguyenfts/Example-sentiment-comment.git

Cloning into 'Example-sentiment-comment'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 19 (delta 5), reused 13 (delta 2), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [0]:
!ls

basedline_model.py		 README.md		  submission.csv
Example-sentiment-comment	 sample.csv		  test.crash
Example_Sentiment_Comment.ipynb  sentiment_comment.ipynb  train.crash


In [0]:
%cd Example-sentiment-comment
!ls

/content/Example-sentiment-comment
basedline_model.py		 sample.csv		  test.crash
Example_Sentiment_Comment.ipynb  sentiment_comment.ipynb  train.crash
README.md			 submission.csv


**2. Import thư viện**

In [0]:
import pandas as pd
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

**3. Xử Lý data**

In [0]:
class DataSource(object):

    def _load_raw_data(self, filename, is_train=True):

        a = []
        b = []

        regex = 'train_'
        if not is_train:
            regex = 'test_'

        with open(filename, 'r') as file:
            for line in file:
                if regex in line:
                    b.append(a)
                    a = [line]
                elif line != '\n':
                    a.append(line)

        b.append(a)

        return b[1:]

    def _create_row(self, sample, is_train=True, dont_remove_space=True):

        d = {}
        d['id'] = sample[0].replace('\n', '')
        review = ""

        if is_train:
            for clause in sample[1:-1]:
                review += clause.replace('\n', '').strip()
            d['label'] = int(sample[-1].replace('\n', ''))
        else:
            for clause in sample[1:]:
                review += clause.replace('\n', '').strip()

        d['review'] = review

        return d

    def load_data(self, filename, is_train=True):

        raw_data = self._load_raw_data(filename, is_train)
        lst = []

        for row in raw_data:
            lst.append(self._create_row(row, is_train))

        return lst


**4. Tiền xử lý data**

In [0]:
ds = DataSource()
train_data = pd.DataFrame(ds.load_data('train.crash'))
test_data = pd.DataFrame(ds.load_data('test.crash', is_train=False))

vectorizer = TfidfVectorizer(max_features=100000, ngram_range=(1, 3))

x_train, x_val, y_train, y_val = train_test_split(train_data.review, train_data.label, test_size=0.2,
                                                  random_state=42)

vectorizer.fit(x_train)
x_tfidf_train = vectorizer.transform(x_train)
x_tfidf_val = vectorizer.transform(x_val)


**5. Build model**

In [0]:
model = LogisticRegression()
print("Train model.......")
sentiment_fit = model.fit(x_tfidf_train, y_train)
print("Predict ......")
y_pred = sentiment_fit.predict(x_tfidf_val)
accuracy = accuracy_score(y_val, y_pred)
print("accuracy score: {0:.2f}%".format(accuracy * 100))

Train model.......


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Predict ......
accuracy score: 88.63%
